## 作業 : (Kaggle)鐵達尼生存預測  

### [作業目標]
- 試著完成三種不同特徵類型的三種資料操作, 觀察結果
- 思考一下, 這三種特徵類型, 哪一種應該最複雜/最難處理  

### [作業重點]
- 完成剩餘的八種 類型 x 操作組合  
- 思考何種特徵類型, 應該最複雜

In [1]:
# 載入基本套件
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# 讀取訓練與測試資料
df_train = pd.read_csv("data/titanic_train.csv")
df_test = pd.read_csv("data/titanic_test.csv")
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
# 重組資料成為訓練 / 預測用格式
train_Y = df_train["Survived"]
ids = df_test["PassengerId"]
df_train = df_train.drop(["PassengerId","Survived"], axis=1)
df_test = df_test.drop(["PassengerId"], axis=1)
df = pd.concat([df_train,df_test]).reset_index(drop=True)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# 秀出資料欄位的類型與數量
dtype_df = df.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
dtype_df = dtype_df.groupby("Column Type").count().reset_index()
dtype_df

,Column Type,Count
0,int64,3
1,float64,2
2,object,5


In [4]:
# 確定只有 int64,float64,object三種類型後,分別將欄位名稱存於三個 list 中
int_features = []
float_features = []
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64':
        float_features.append(feature)
    elif dtype == 'int64':
        int_features.append(feature)
    else:
        object_features.append(feature)
print(f'{len(int_features)} Integer Features : {int_features}')
print(f'{len(float_features)} Float Features : {float_features}')
print(f'{len(object_features)} Object Features : {object_features}')

3 Integer Features : ['Pclass', 'SibSp', 'Parch']
2 Float Features : ['Age', 'Fare']
5 Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']


## 作業1 
- 試著執行作業程式，觀察三種類型(int / float / object)的欄位分別進行(平均 mean / 最大值 Max / 相異值 nunique)中的九次操作會有那些問題?  
並試著解釋那些發生Error的程式區塊的原因?  
- <font color=#0000FF>對object"物件"進行mean()、max()等會有問題，因不賦有數值意義，無法進行數值上的運算</font>

## 作業2
- 思考一下，試著舉出今天五種類型以外的一種或多種資料類型，你舉出的新類型是否可以歸在三大類中的某些大類?  
所以三大類特徵中，哪一大類處理起來應該最複雜?
- <font color=#0000FF>(1) 文本類型, 無法將其歸類在三大類型  
(2) 類別型特徵, 因為需要思考如何轉換為數值, 會影響後續的模型</font>

In [5]:
# EX: 整數 (int)特徵取平均(mean)
df[int_features].mean()

Pclass    2.294882
SibSp     0.498854
Parch     0.385027
dtype: float64

In [6]:
# 請依序列出 三種特徵類型 (int / float / object) x 三種方法 (平均 mean / 最大值 Max / 相異值 nunique) 的其餘操作
type_data = {"Int Features":int_features, "Float Features":float_features, "Object Features":object_features}
for typ in type_data:
    print("{} X Mean(): \n\n{}\n====================".format(typ, df[type_data[typ]].mean()))
    print("{} X Max(): \n\n{}\n====================".format(typ, df[type_data[typ]].max()))
    print("{} X Nuique(): \n\n{}\n====================".format(typ, df[type_data[typ]].nunique()))

Int Features X Mean(): 

Pclass    2.294882
SibSp     0.498854
Parch     0.385027
dtype: float64
Int Features X Max(): 

Pclass    3
SibSp     8
Parch     9
dtype: int64
Int Features X Nuique(): 

Pclass    3
SibSp     7
Parch     8
dtype: int64
Float Features X Mean(): 

Age     29.881138
Fare    33.295479
dtype: float64
Float Features X Max(): 

Age      80.0000
Fare    512.3292
dtype: float64
Float Features X Nuique(): 

Age      98
Fare    281
dtype: int64
Object Features X Mean(): 

Series([], dtype: float64)
Object Features X Max(): 

Name      van Melkebeke, Mr. Philemon
Sex                              male
Ticket                      WE/P 5735
dtype: object
Object Features X Nuique(): 

Name        1307
Sex            2
Ticket       929
Cabin        186
Embarked       3
dtype: int64
